In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time
import pandas as pd
from helper import get_penmap
import warnings
warnings.simplefilter('ignore')

Using TensorFlow backend.


In [2]:
def compute_dist(param1,param2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def distance_distribution(features):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                        except:
                            
                            pass
     
    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
   # print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

def pen_distribution(features,a,pen_out):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        if(k in a[pen_out]):
            for i in ['1','2']: 
                for n in ['1','2']:
                    for j in ['1','2','3','4','5','6','7','8'] :
                        for m in ['1','2','3','4','5','6','7','8'] :
                            try:
                                if(i==n and j==m):
                                    continue    
                                elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                    same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])

                                elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                    diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                            except:

                                pass

    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
    return same_pens,diff_pens

In [3]:
def distance_btw_distributions(feature,a):
    d = []
    same_pens,diff_pens = distance_distribution(feature)
    D = dist(same_pens,diff_pens,1465,1456)
    for j in range(1,15):
        same_pens_i,diff_pens_i= pen_distribution(feature,a,j)
        d1 = len(same_pens_i)/2930*dist(same_pens_i,same_pens,len(same_pens_i),1465)
        d2 = len(diff_pens_i)/2912*dist(diff_pens_i,diff_pens,len(diff_pens_i),1456)
        d.append(d1+d2)
        
    return D-sum(d)

In [4]:
def load_parameters(path,n_param):
    hist_Y = {}
    hist_Cb = {}
    hist_Cr = {}
    a = pd.read_csv(path)
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][0:n_param] 
                    hist_Cb['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][n_param:2*n_param]
                    hist_Cr['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][2*n_param:3*n_param]
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cb['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cr['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    
                except:
                    break
    return hist_Y,hist_Cb,hist_Cr

def load_parameters_dominant(path,n1,n2,n3):
    hist_Y = {}
    hist_Cb = {}
    hist_Cr = {}
    a = pd.read_csv(path)
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][0:n1] 
                    hist_Cb['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][n1:n1+n2]
                    hist_Cr['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][n1+n2:n1+n2+n3]
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cb['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cr['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    
                except:
                    break
    return hist_Y,hist_Cb,hist_Cr

In [5]:
def concat2(h1,h2):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

def concat3(h1,h2,h3):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j],h3['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

def concat4(h1,h2,h3,h4):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j],h3['c'+str(k)+'_'+i+'_'+j],h4['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

In [6]:
a = get_penmap()

In [7]:
dir = '/home/mohit/Desktop/featuresUpdated/'
path = {}

path[1] = 'LBP_riu2.csv'
path[2] = 'LBP_default.csv'

path[3] = 'Dominant_LBP.csv'

path[4] = 'CSLBP_ri_T=20,10,10.csv'
path[5] = 'CSLBP_default_T=20,10,10.csv'

path[6] = 'ELBP_ni_ci_riu2.csv'
path[7] = 'ELBP_ni_ci_default.csv'
path[8] = 'ELBP_rd_ci_riu2.csv'
path[9] = 'ELBP_rd_ci_default.csv'

path[10] = 'CLBP_S_c_riu2.csv'
path[11] = 'CLBP_S_c_default.csv'
path[12] = 'CLBP_M_c_riu2.csv'
path[13] = 'CLBP_M_c_default.csv'

path[14] = 'OCS-CLBP_S_c_riu2.csv'
path[15] = 'OCS-CLBP_S_c_default.csv'
path[16] = 'OCS-CLBP_M_c_riu2.csv'
path[17] = 'OCS-CLBP_M_c_default.csv'


l={}  #lengths of feature vectors for a single channel
l[1] = 10
l[2] = 256

l[3] = 7,10,9

l[4] = 6
l[5] = 16

l[6] = 18
l[7] = 512
l[8] = 20
l[9] = 512

l[10] = 20
l[11] = 512
l[12] = 20
l[13] = 512

l[14] = 12
l[15] = 32
l[16] = 12
l[17] = 32


# Loading all features in respective channels

In [8]:
y  = {}
cb = {}
cr = {}
for i in [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17]:
        y[i],cb[i],cr[i] = load_parameters(dir+path[i],l[i])
y[3],cb[3],cr[3] = load_parameters_dominant(dir+path[3],l[3][0],l[3][1],l[3][2])


### so, we have 17 variants in all 3 channels. first we'll compare these 17*3 lbps and selects the best lbp in any one channel. that will be our starting point in the process

In [9]:
Distances = {}
for i in range(1,18):
    dy  = distance_btw_distributions(y[i],a)
    dcb = distance_btw_distributions(cb[i],a)
    dcr = distance_btw_distributions(cr[i],a)
    
    Distances[str(i)+'y'] = dy
    Distances[str(i)+'cb'] = dcb
    Distances[str(i)+'cr'] = dcr
    
    print('Features {} '.format(i))
    print('Distance : Y {} Cb {} Cr {}'.format(dy,dcb,dcr))

Features 1 
Distance : Y 17.48021859079082 Cb 14.953443329934558 Cr 6.434224978618882
Features 2 
Distance : Y 16.00901758794798 Cb 18.828429029884802 Cr 13.995547143521438
Features 3 
Distance : Y 15.815963906840398 Cb 21.467090675535125 Cr 13.035956935874562
Features 4 
Distance : Y 12.229946790304208 Cb 19.714819693937113 Cr 10.493006879159475
Features 5 
Distance : Y 14.839570105387924 Cb 18.96793880362152 Cr 13.906832785657032
Features 6 
Distance : Y 22.881688357890006 Cb 21.507676846172526 Cr 10.318795169563192
Features 7 
Distance : Y 17.70335521433232 Cb 22.062883490958065 Cr 13.864863821585905
Features 8 
Distance : Y 16.317338794024348 Cb 22.909956317368007 Cr 12.545552944492336
Features 9 
Distance : Y 18.875457470813913 Cb 14.777003855969156 Cr 7.341387255248199
Features 10 
Distance : Y 25.304412703348724 Cb 19.987516427486426 Cr 10.598955548960452
Features 11 
Distance : Y 19.04668981486516 Cb 21.134009850123554 Cr 15.112249023560352
Features 12 
Distance : Y 18.55211848

In [16]:
sorted_by_value = sorted(Distances.items(), key=lambda kv: kv[1],reverse =True)

In [17]:
sorted_by_value

[('10y', 25.304412703348724),
 ('14y', 22.914317658177836),
 ('8cb', 22.909956317368014),
 ('6y', 22.88168835789001),
 ('7cb', 22.062883490958065),
 ('15y', 21.743953219514026),
 ('6cb', 21.507676846172526),
 ('3cb', 21.467090675535125),
 ('11cb', 21.134009850123554),
 ('15cb', 20.233444445501842),
 ('17y', 20.133702516769908),
 ('10cb', 19.98751642748642),
 ('4cb', 19.714819693937116),
 ('11y', 19.04668981486514),
 ('5cb', 18.96793880362151),
 ('9y', 18.875457470813913),
 ('14cb', 18.829150370487078),
 ('2cb', 18.828429029884802),
 ('12y', 18.55211848980681),
 ('13y', 17.70562026857813),
 ('7y', 17.70335521433232),
 ('1y', 17.48021859079082),
 ('17cr', 17.000815023000527),
 ('8y', 16.31733879402435),
 ('16y', 16.248107526513664),
 ('2y', 16.009017587947966),
 ('3y', 15.815963906840405),
 ('15cr', 15.652494249301794),
 ('11cr', 15.112249023560356),
 ('1cb', 14.953443329934556),
 ('5y', 14.839570105387924),
 ('9cb', 14.777003855969152),
 ('13cb', 14.24459118464218),
 ('2cr', 13.99554714

### SO, We will try  starting points in every lbp group based on the above rankings.
### consider starting point -> 10y 

In [20]:
# with 10y we can select a feature among[10cb,10cr,12y]
fe1 = {}
Distances1 = {}

fe1['10y10cb'] = concat2(y[10],cb[10])
fe1['10y10cr'] = concat2(y[10],cr[10])
fe1['10y12y'] = concat2(y[10],y[12])
for keys in fe1.keys():
    Distances1[keys] = distance_btw_distributions(fe1[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances1[keys]-Distances['10y']))

Features 10y12y 
Distance : -3.597105020838285 
Features 10y10cr 
Distance : -6.723250254103082 
Features 10y10cb 
Distance : 1.7461365572832008 


In [21]:
# so we will select 10cb and with [10y,10cb], we can select among [10cr,12y|12cb]

fe1['10y10cb10cr'] = concat3(y[10],cb[10],cr[10])
fe1['10y10cb12y12cb'] = concat4(y[10],cb[10],y[12],cb[12])

for keys in fe1.keys():
    Distances1[keys] = distance_btw_distributions(fe1[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances1[keys]-Distances['10y']))

Features 10y10cb10cr 
Distance : -3.554053846495293 
Features 10y12y 
Distance : -3.597105020838285 
Features 10y10cr 
Distance : -6.723250254103082 
Features 10y10cb 
Distance : 1.7461365572832008 
Features 10y10cb12y12cb 
Distance : -1.6135257693809955 


In [22]:
Distances1

{'10y10cb': 27.050549260631925,
 '10y10cb10cr': 21.75035885685343,
 '10y10cb12y12cb': 23.69088693396773,
 '10y10cr': 18.581162449245642,
 '10y12y': 21.70730768251044}

### So starting from 10y, our feature subset is [10y,10cb] with distance = 27.05

### now we will try another starting point i.e. 14y

In [27]:
# with 14y we can select a feature among[14cb,14cr,16y]
fe2 = {}
Distances2 = {}

fe2['14y14cb'] = concat2(y[14],cb[14])
fe2['14y14cr'] = concat2(y[14],cr[14])
fe2['14y16y'] = concat2(y[14],y[16])
for keys in fe2.keys():
    Distances2[keys] = distance_btw_distributions(fe2[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances2[keys]-Distances['14y']))

Features 14y16y 
Distance : -3.193632279340971 
Features 14y14cr 
Distance : -3.9187087410124875 
Features 14y14cb 
Distance : 4.72750769362905 


In [28]:
# so we will select 14cb and with [14y,14cb], we can select among [14cr,16y|16cb]

fe2['14y14cb14cr'] = concat3(y[14],cb[14],cr[14])
fe2['14y14cb16y16cb'] = concat4(y[14],cb[14],y[16],cb[16])

for keys in fe2.keys():
    Distances2[keys] = distance_btw_distributions(fe2[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances2[keys] -Distances['14y']))

Features 14y14cb14cr 
Distance : 2.209435495434249 
Features 14y14cb16y16cb 
Distance : -0.7549442566648494 
Features 14y16y 
Distance : -3.193632279340971 
Features 14y14cr 
Distance : -3.9187087410124875 
Features 14y14cb 
Distance : 4.72750769362905 


In [29]:
Distances4

{'14y14cb': 27.641825351806887,
 '14y14cb14cr': 25.123753153612085,
 '14y14cb16y16cb': 22.159373401512987,
 '14y14cr': 18.99560891716535,
 '14y16y': 19.720685378836865}

### So starting from 14y our feature subset is [14y,14cb]  with a distance = 27.64

### Now we will try starting point -> 8cb

In [30]:
# with 8cb we can select among [8cr,8y,6cb]
fe3 = {}
Distances3 = {}
fe3['8cb8y'] = concat2(cb[8],y[8])
fe3['8cb8cr'] = concat2(cb[8],cr[8])
fe3['8cb6cb'] = concat2(cb[8],cb[6])

for keys in fe3.keys():
    Distances3[keys] = distance_btw_distributions(fe3[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances3[keys]-Distances['8cb']))

Features 8cb8cr 
Distance : -1.5439921671541939 
Features 8cb8y 
Distance : -0.925286212564874 
Features 8cb6cb 
Distance : 1.0444180098267069 


In [31]:
# so we will select 8cb and with [8cb,6cb], we can select among [6y|8y,6cr|8cr]

fe3['6cb6cr8cb8cr'] = concat4(cb[6],cr[6],cb[8],cr[8])
fe3['6y6cb8y8cb'] = concat4(y[6],cb[6],y[8],cb[8])

for keys in fe3.keys():
    Distances3[keys] = distance_btw_distributions(fe3[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances3[keys]-Distances['8cb']))

Features 6y6cb8y8cb 
Distance : 4.754728023522361 
Features 8cb8cr 
Distance : -1.5439921671541939 
Features 8cb8y 
Distance : -0.925286212564874 
Features 6cb6cr8cb8cr 
Distance : -2.212293664908003 
Features 8cb6cb 
Distance : 1.0444180098267069 


In [33]:
Distances3

{'6cb6cr8cb8cr': 20.69766265246001,
 '6y6cb8y8cb': 27.664684340890375,
 '8cb6cb': 23.95437432719472,
 '8cb8cr': 21.36596415021382,
 '8cb8y': 21.98467010480314}

### So starting from 8cb, our feature subset is [6y,6cb,8y,8cb] with a distance of 27.66

### now we will try another starting point i.e. 3cb

In [34]:
# with 3cb we can select among [3y,3cr,3y|3cr]
fe4 = {}
Distances4 = {}
fe4['3cb3y'] = concat2(cb[3],y[3])
fe4['3cb3cr'] = concat2(cb[3],cr[3])
fe4['3y3cb3cr'] = concat3(y[3],cb[3],cr[3])

for keys in fe4.keys():
    Distances4[keys] = distance_btw_distributions(fe4[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances4[keys]-Distances['3cb']))

Features 3cb3cr 
Distance : 1.2124053218119997 
Features 3y3cb3cr 
Distance : 3.901014918106359 
Features 3cb3y 
Distance : 5.377715052556184 


In [35]:
Distances4

{'3cb3cr': 22.679495997347125,
 '3cb3y': 26.84480572809131,
 '3y3cb3cr': 25.368105593641484}

### So starting from 3cb, our feature subset is [3y,3cb] with a distance of 26.84

### now we will try another starting point i.e. 4cb

In [36]:
# with 4cb we can select among [4y,4cr,4y|4cr]
fe5 = {}
Distances5 = {}
fe5['4y4cb'] = concat2(cb[4],y[4])
fe5['4cb4cr'] = concat2(cb[4],cr[4])
fe5['4y4cb4cr'] = concat3(y[4],cb[4],cr[4])

for keys in fe5.keys():
    Distances5[keys] = distance_btw_distributions(fe5[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances5[keys]-Distances['4cb']))

Features 4cb4cr 
Distance : 4.813951090881197 
Features 4y4cb 
Distance : 5.292752518015554 
Features 4y4cb4cr 
Distance : 6.362632014932746 


In [37]:
Distances5

{'4cb4cr': 24.528770784818313,
 '4y4cb': 25.00757221195267,
 '4y4cb4cr': 26.077451708869862}

### So starting from 4cb our feature subset is [4y,4cb,4cr]  with a distance = 26.07

### Now we will try starting point -> 2cb

In [39]:
# with 2cb we can select among [2y,2cr,2y|2cr]
fe6 = {}
Distances6 = {}
fe6['2y2cb'] = concat2(cb[2],y[2])
fe6['2cb2cr'] = concat2(cb[2],cr[2])
fe6['2y2cb2cr'] = concat3(y[2],cb[2],cr[2])

for keys in fe6.keys():
    Distances6[keys] = distance_btw_distributions(fe6[keys],a)
    
    print('Features {} '.format(keys))
    print('Distance : {} '.format(Distances6[keys]-Distances['2cb']))

Features 2y2cb 
Distance : -0.02857112954925256 
Features 2y2cb2cr 
Distance : -0.6142656736897045 
Features 2cb2cr 
Distance : -1.0943095824451703 


In [40]:
Distances6

{'2cb2cr': 17.73411944743963,
 '2y2cb': 18.79985790033555,
 '2y2cb2cr': 18.214163356195098}

### so starting from 2cb no more feature will be selected and hence distance is 18.82

## so we tried starting from all 6 variants of lbp and here are the distances!
```
 default LBP -> 18.82
 Dominant    -> 26.84
 CSLBP       -> 26.07
 ELBP        -> 27.66
 CLBP        -> 27.05
 OCS_CLBP    -> 27.64
 ```

# After this feature selection process, selected features are [6Y,6Cb,8Y,8Cb]

In [42]:
a = pd.DataFrame(fe3['6y6cb8y8cb'])
a

,c100_1_1,c100_1_2,c100_1_3,c100_1_4,c100_2_1,c100_2_2,c100_2_3,c101_1_1,c101_1_2,c101_1_3,...,c99_2_3,c9_1_1,c9_1_2,c9_1_3,c9_1_4,c9_1_5,c9_1_6,c9_1_7,c9_2_1,c9_2_2
0,0.000000,0.000000,0.000000,0.000000,0.000354,0.000000,0.000000,0.000319,0.000000,0.000562,...,0.000000,0.000000,0.000000,0.000000,0.000575,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000387,0.000000,0.000000,0.000304,0.000000,0.000000,0.000533,0.000638,0.002341,0.000562,...,0.000737,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.003484,0.004021,0.002126,0.004562,0.001416,0.001546,0.002131,0.000638,0.001171,0.003371,...,0.000000,0.000000,0.000665,0.000000,0.000575,0.000596,0.002343,0.000000,0.000000,0.000803
3,0.020906,0.024698,0.020553,0.023723,0.012389,0.010823,0.012786,0.019127,0.014436,0.015169,...,0.013274,0.008322,0.013298,0.010595,0.023577,0.014009,0.019918,0.019730,0.007212,0.015261
4,0.039876,0.040207,0.048193,0.045925,0.050265,0.042134,0.059670,0.072362,0.066329,0.061798,...,0.035398,0.041609,0.051197,0.076610,0.089132,0.098063,0.080844,0.084112,0.073317,0.064659
5,0.134727,0.143021,0.170801,0.163625,0.248142,0.270584,0.240277,0.240038,0.203277,0.232022,...,0.295723,0.162275,0.204787,0.195599,0.202990,0.197019,0.216169,0.204915,0.259615,0.292369
6,0.102207,0.085009,0.078668,0.081509,0.099823,0.100503,0.104422,0.103921,0.114709,0.100562,...,0.101032,0.110957,0.101729,0.070090,0.047729,0.056334,0.064441,0.066113,0.066106,0.059438
7,0.024390,0.020103,0.007796,0.016119,0.010973,0.010050,0.019180,0.023589,0.025361,0.017978,...,0.017699,0.029126,0.009973,0.004890,0.002875,0.005067,0.002343,0.006923,0.003606,0.003213
8,0.223384,0.211373,0.194897,0.204684,0.183363,0.169308,0.171550,0.142812,0.177136,0.173034,...,0.152655,0.156727,0.139628,0.127954,0.126509,0.116841,0.123609,0.118726,0.127404,0.115261
9,0.000000,0.000000,0.000000,0.000000,0.001062,0.000000,0.001066,0.000000,0.001951,0.000562,...,0.000000,0.001387,0.000000,0.000000,0.000000,0.000596,0.000000,0.000000,0.000000,0.000000


In [43]:
a.to_csv(dir+'LBP_selected_features.csv')